# n-puzzle problem

In [219]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import functools

In [220]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

## Utility functions

In [221]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

In [222]:
class State :
    def __init__(self, matrix: np.ndarray):
        self.matrix = matrix
        self.g = float('inf') #cost root to current
        self.f = float('inf') #cost root to goal
        self.h = float('inf') #heuristic : estimated cost from current to goal

    #check if the current state contains a matrix that represents the solution
    def is_goal(self):
        return np.array_equal(self.matrix, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))

class Path:
    def __init__(self):
        self.path = []

    def push(self, item):
        self.path.append(item)

    def pop(self):
        if self.is_empty():
            raise IndexError("path is empty")
        return self.path.pop()

    def last(self):
        if self.is_empty():
            raise IndexError("path is empty")
        return self.path[-1]

    def is_empty(self):
        return len(self.path) == 0

In [223]:
def cost_root_to_current(path : list):
    return len(path)

def manhattan_heuristic(state:State,goal_state:State)->int:
    distance = 0
    for elem in range(1,PUZZLE_DIM**2):
        start_coords = np.where(state.matrix == elem)
        end_coords = np.where(goal_state.matrix == elem)
        distance+=  abs(start_coords[0]-end_coords[0])+abs(start_coords[1]-end_coords[1])
    return distance


def available_actions(state: State) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state.matrix == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


@counter
def do_action(state: State, action: 'Action') -> State:
    new_state = state.matrix.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return State(new_state)

def print_solution(solution : tuple[Path,int]):
    if solution[0] == None:
        print("No solution was found")
    else:
        print(f"Solution was found in {len(solution[0].path)} steps with cost of {do_action.calls}")
        print("Solution path:")
        for state in solution[0].path:
            print(state.matrix)
            print()


In [224]:
# test manhattan heuristic
state = State(test)
# goal_state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))
# print(state.matrix)
# print(goal_state.matrix)

# print(manhattan_heuristic(state,goal_state))

# print(available_actions(state))
# choose action
# lowest_heuristic = 100
# best_action = None
# for action in available_actions(state):
#     new_state = do_action(state.matrix,action)
#     heuristic = manhattan_heuristic(State(new_state),goal_state)
#     print(heuristic)
#     if heuristic < lowest_heuristic:
#         lowest_heuristic = heuristic
#         best_action = action

# print(best_action)

# new_state = do_action(state.matrix,best_action)
print(new_state)

[[4 5 3]
 [0 2 6]
 [7 8 1]]


In [225]:
RANDOMIZE_STEPS = 100_000
state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state.matrix

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[3, 6, 7],
       [2, 0, 1],
       [4, 8, 5]])

## IDA*

In [226]:
# takes as parameter a state and a heuristic function
def ida_star(initial_state : State, goal_state : State, heuristic):
    bound = heuristic(initial_state,goal_state)
    path = Path()
    path.push(initial_state)

    while True:
        t = search(path, 0, bound, heuristic, goal_state)
        if t < 0: ## FOUND = -1
            return (path, bound)
        elif t == float('inf'):
            return (None, float('inf')) 
        else: # t belongs to [0,inf[
            bound = t
           # path.push(t)

def search(path : Path, g : float, bound : int, h, goal_state : State) :
    node = path.last()
    f = g + h(node, goal_state)
    if f > bound :
        return f
    if node.is_goal():
        return -1 # FOUND
    
    min = float('inf')

    for action in available_actions(node):
        successor = do_action(node, action)
        if successor not in path.path:
            path.push(successor)
            t = search(path,cost_root_to_current(path.path), bound, h, goal_state)
            if t == -1:
                return -1
            if t < min :
                min = t
            path.pop()
        



    return min

In [ ]:
# easy solvable solution
#state = State(np.array([np.array([1,8,2]),np.array([0,4,3]),np.array([7,6,5])]))
#state = State(np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])]))
state = np.array([[13,  9,  0, 10, 19],
[ 3, 21, 14,  5,  8],
[22, 16,  4, 24, 18],
[ 6,  2, 11,  1, 20],
[ 7, 15,23,12,17]])
state = State(state)


goal_state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))
print(state.matrix)
print(goal_state.matrix)

solution = ida_star(state,goal_state, manhattan_heuristic)
print_solution(solution)




[[0 2 1]
 [3 7 5]
 [8 6 4]]
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Solution was found in 23 steps with cost of 255164
Solution path:
[[0 2 1]
 [3 7 5]
 [8 6 4]]

[[3 2 1]
 [0 7 5]
 [8 6 4]]

[[3 2 1]
 [7 0 5]
 [8 6 4]]

[[3 0 1]
 [7 2 5]
 [8 6 4]]

[[3 1 0]
 [7 2 5]
 [8 6 4]]

[[3 1 5]
 [7 2 0]
 [8 6 4]]

[[3 1 5]
 [7 2 4]
 [8 6 0]]

[[3 1 5]
 [7 2 4]
 [8 0 6]]

[[3 1 5]
 [7 2 4]
 [0 8 6]]

[[3 1 5]
 [0 2 4]
 [7 8 6]]

[[3 1 5]
 [2 0 4]
 [7 8 6]]

[[3 0 5]
 [2 1 4]
 [7 8 6]]

[[0 3 5]
 [2 1 4]
 [7 8 6]]

[[2 3 5]
 [0 1 4]
 [7 8 6]]

[[2 3 5]
 [1 0 4]
 [7 8 6]]

[[2 3 5]
 [1 4 0]
 [7 8 6]]

[[2 3 0]
 [1 4 5]
 [7 8 6]]

[[2 0 3]
 [1 4 5]
 [7 8 6]]

[[0 2 3]
 [1 4 5]
 [7 8 6]]

[[1 2 3]
 [0 4 5]
 [7 8 6]]

[[1 2 3]
 [4 0 5]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

